In [1]:
%reload_ext autoreload
%autoreload 2
import os, sys
import numpy as np
from collections import OrderedDict
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)
from utilities.sqlcontroller import SqlController
from utilities.utilities_cvat_neuroglancer import get_structure_number
atlas_name = 'atlas'
sqlController = SqlController(atlas_name)
from utilities.file_location import DATA_PATH

Connecting dklab@192.168.1.12:3306
No histology for atlas
No scan run for atlas


In [2]:
# static variables used by both methods
col_length = 1000
row_length = 1000
z_length = 300
atlas_box_size=(row_length, col_length, z_length)
atlas_box_scales=(10, 10, 20)
atlas_raw_scale=10

In [3]:
# calculate the centers of mass for the atlas with the existing algorithm
atlas_centers = sqlController.get_centers_dict(atlas_name)
# with no sql connection, comment line above and uncomment the next 4 lines
#atlas_centers = {}
#atlas_centers['DC_L'] = [83,-43,-215]
#atlas_centers['DC_R'] = [83,-43,215]
#atlas_centers['SC'] = [-128,-241,3]
atlas_box_scales = np.array(atlas_box_scales)
atlas_box_size = np.array(atlas_box_size)
atlas_box_center = atlas_box_size / 2
com_calc = OrderedDict()
for structure, origin in atlas_centers.items():
    x,y,section = atlas_box_center + np.array(origin) * atlas_raw_scale / atlas_box_scales
    com_calc[structure] = [x,y,section]

In [4]:
# get the centers of mass from the existing structures
# This is the process that is used to create the actual atlas we see in neuroglancer,
# to see the entire code for creating an atlas, see:
# https://github.com/eddyod/pipeline_utility/blob/master/utilities/atlas/create_atlas_volume.py
ATLAS_PATH = os.path.join(DATA_PATH, 'atlas_data', atlas_name)
ORIGIN_PATH = os.path.join(ATLAS_PATH, 'origin')
VOLUME_PATH = os.path.join(ATLAS_PATH, 'structure')
origin_files = sorted(os.listdir(ORIGIN_PATH))
volume_files = sorted(os.listdir(VOLUME_PATH))

structure_volume_origin = {}
for volume_filename, origin_filename in zip(volume_files, origin_files):
    structure = os.path.splitext(volume_filename)[0]
    origin = np.loadtxt(os.path.join(ORIGIN_PATH, origin_filename))
    volume_path = os.path.join(VOLUME_PATH, volume_filename)
    volume = np.load(volume_path)
    volume = np.rot90(volume, axes=(0, 1))
    volume = np.flip(volume, axis=0)
    volume = volume.astype(np.uint8)
    structure_volume_origin[structure] = (volume, origin)

common_keys = com_calc.keys() & structure_volume_origin.keys()
com_existing = OrderedDict()
for structure, (volume, origin) in sorted(structure_volume_origin.items()):
    if structure not in common_keys:
        continue

    x, y, z = origin
    x_start = x + col_length / 2
    y_start = y + row_length / 2
    z_start = z / 2 + z_length / 2
    x_end = x_start + volume.shape[0]
    y_end = y_start + volume.shape[1]
    z_end = z_start + (volume.shape[2] + 1) / 2

    midx = ((x_start + x_end) / 2)
    midy = ((y_start + y_end) / 2)
    midz = (z_start + z_end) / 2
    com_existing[structure] = [midx,midy,midz]

In [5]:
# look at the difference between the two methods. Most are very close,
# but some like DC_L and DC_R vary a lot in the section
for (kc,vc),(ke,ve) in zip(com_calc.items(), com_existing.items()):
    arrc = np.array(vc)
    arre = np.array(ve)
    if kc == ke:
        diffed = np.array(arrc-arre).astype(np.int)
        print(kc,end="\t")
        print(diffed)


10N_L	[ -9 -15   0]
10N_R	[ -9 -15   0]
12N	[-6 -3  0]
3N_L	[-1 -1  0]
3N_R	[-1 -1  0]
4N_L	[ 0 -1  0]
4N_R	[ 0 -1  0]
5N_L	[2 0 0]
5N_R	[ 2  0 -1]
6N_L	[1 0 0]
6N_R	[ 1  0 -1]
AP	[ 0 -2  0]
Amb_L	[-4 -1  0]
Amb_R	[-4 -1  0]
DC_L	[  8 -11 -14]
DC_R	[  8 -11  13]
IC	[39 -3  0]
LC_L	[-3 -4  0]
LC_R	[-3 -4  0]
LRt_L	[-3  7  0]
LRt_R	[-3  7 -1]
PBG_L	[-5  0  0]
PBG_R	[-5  0  0]
Pn_L	[-8  1  3]
Pn_R	[-8  1 -4]
RtTg	[-21  -7   0]
SC	[  1 -11   0]
SNC_L	[-6  6 -1]
SNC_R	[-6  6  0]
SNR_L	[ 3  7 -1]
SNR_R	[3 7 0]
Sp5C_L	[-18  -7  -6]
Sp5C_R	[-18  -7   5]
Sp5I_L	[-7 -6  0]
Sp5I_R	[-7 -6 -1]
Sp5O_L	[2 0 0]
Sp5O_R	[ 2  0 -1]
Tz_L	[-3  0 -1]
Tz_R	[-3  0  0]
VCA_L	[  0 -10  -3]
VCA_R	[  0 -10   2]
VCP_L	[-2  3 -3]
VCP_R	[-2  3  2]
VLL_L	[-12  -1  -2]
VLL_R	[-12  -1   1]
